## Bedrock model integration with Langchain Agents 
### Financial stock analyser example 

**Reference**: https://medium.com/@pranav.kushare2001/an-ai-based-stock-analyzer-using-llm-and-langchain-7f8a62cbcaaa

**Note:** The code in the notebook is based on the medium post, the link for which is provided as reference above. 

Certain applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Agent interface enables such flexibility for these applications. An agent has availability to a range of resources and selects which ones to utilize based on the user input. Agents are capable of using multiple tools and utilizing the output of one tool as the input for the next.  

There are two primary categories of agents:

- Action agents: At each interval, determine the subsequent action utilizing the outputs of all previous actions. 
- Plan-and-execute agents: Determine the complete order of actions initially, then implement them all without updating the plan.

In this notebook, we will demonstrate the use `Zero-shot ReAct` agent based on [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) framework to select the appropriate tool based exclusively on the tool's description. It requires you provide the description of each tool. 

## Architecture
![](./images/arch-agents.png)

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [ ]:
import json
import os
import sys
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

In [ ]:
from langchain.llms.bedrock import Bedrock
from typing import Optional, List, Any
from langchain.callbacks.manager import CallbackManagerForLLMRun

class BedrockModelWrapper(Bedrock):
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        prompt = "\n\nHuman: " + prompt + "\n\nAssistant:"   ## Satisfy Bedrock-Claude prompt requirements
        return super()._call(prompt, stop, run_manager, **kwargs)

In [ ]:

model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}
llm = BedrockModelWrapper(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs=model_parameter, verbose=True)


## Part 1 - Prepare dataset in SQLite Database

In [ ]:
## database
stock_ticker_data=[ 
    {
        "symbol" : "PRAA",
        "name" : "PRA Group, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "AMZN",
        "name" : "Amazon.com, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "TSLA",
        "name" : "Tesla Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "PAAS",
        "name" : "Pan American Silver Corp.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "PAAC",
        "name" : "Proficient Alpha Acquisition Corp.",
        "currency" : "USD",
        "stockExchange" : "NasdaqCM",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "RYAAY",
        "name" : "Ryanair Holdings plc",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "MPAA",
        "name" : "Motorcar Parts of America, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "STAA",
        "name" : "STAAR Surgical Company",
        "currency" : "USD",
        "stockExchange" : "NasdaqGM",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "RBCAA",
        "name" : "Republic Bancorp, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "AABA",
        "name" : "Altaba Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGS",
        "exchangeShortName" : "NASDAQ"    
    }, 
    {
        "symbol" : "AAXJ",
        "name" : "iShares MSCI All Country Asia ex Japan ETF",
        "currency" : "USD",
        "stockExchange" : "NasdaqGM",
        "exchangeShortName" : "NASDAQ"
    }, 
    {
        "symbol" : "ZNWAA",
        "name" : "Zion Oil & Gas, Inc.",
        "currency" : "USD",
        "stockExchange" : "NasdaqGM",
        "exchangeShortName" : "NASDAQ"
    }
]

In [ ]:
import sqlite3
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except FileExistsError as e:
        print(e)

    return conn

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except RuntimeError as e:
        print(e)

In [ ]:
db_name = "stock_ticker_database.db"
conn = create_connection(db_name)

In [ ]:
create_table_sql = """CREATE TABLE IF NOT EXISTS stock_ticker (
	symbol text PRIMARY KEY,
	name text NOT NULL,
	currency text,
	stockExchange text, 
    exchangeShortName text
);"""


In [ ]:
# create tables
if conn is not None:
    # create projects table
    create_table(conn, create_table_sql)
else:
    print("Error! cannot create the database connection.")


In [ ]:
def insert_data(data):
    for item in data:
        conn.execute("INSERT INTO stock_ticker (symbol, name, currency,stockExchange, exchangeShortName ) VALUES (?, ?, ?, ?,?)", 
                    (item["symbol"], item["name"], item["currency"], item["stockExchange"],item["exchangeShortName"]))
    conn.commit()
    conn.close()

In [ ]:

# uncomment to insert data if the data doesn't exist in the table.
#insert_data(stock_ticker_data)

## Part 2 - Database Tools
A common use of an agent is to look up a record in a database. It would not be practical to include the full database in the context, so you can provide tools that perform actions against the datebase that eliminates hallucinations while maintining the conversational interactions.

### SQL Database Agent
Langchain has a SQL Database agent for demonstrating how to ask questions of a DB to get answers. For details, read this document: https://python.langchain.com/docs/integrations/toolkits/sql_database

The agent will load the schema of the DB into context and generate SQL statements based on natural language questions. The SQL statement is then executed against the database and the results returned.

### Data Agents
While the SQL Database agent is useful for data exploration and generating queries, there are also cases where you want to 
For specific entities, a tool can be created to pull data from the database to provide context next steps in the prompt.

The following example will simulate a DB query for a customer in the customer table. Replace this code with a lookup ib DynamoDB or a relation database.

In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
db = SQLDatabase.from_uri("sqlite:///stock_ticker_database.db")


In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Human: Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
<format>
Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Result of SQLResult only"
</format>
Assistant: Understood, I will use the above format and only provide the answer.

Only use the following tables:
<tables>
CREATE TABLE stock_ticker (
	symbol text PRIMARY KEY,
	name text NOT NULL,
	currency text,
	stockExchange text, 
    exchangeShortName text
)
</tables>

If someone asks for the table stock ticker table, they really mean the stock_ticker table.
<examples>
Question: 
        What is the ticker symbol for Amazon in stock ticker table?
        Params: 
        Company name (name): Amazon
        
SQLQuery:SELECT symbol FROM stock_ticker WHERE name LIKE '%Amazon%'

</examples>

Question: {input}

"""

PROMPT = PromptTemplate(
    input_variables=["input", "dialect"], template=_DEFAULT_TEMPLATE
)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(
    llm, 
    db, 
    verbose=True, 
    return_intermediate_steps=True, 
    prompt=PROMPT, 
    )


In [ ]:
response = db_chain("\n\nHuman: What is the ticker symbol for Tesla in stock ticker table? \n\nAssistant:")


In [ ]:
response['result']

In [ ]:
print(response['intermediate_steps'])

## Part 3 - Helper functions which will be called by ReAct Agent.

In [ ]:
import json
import time
from bs4 import BeautifulSoup
import re
import requests

from langchain.agents import load_tools, AgentType, Tool, initialize_agent
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override() 
import pandas as pd
from datetime import datetime, timedelta

import warnings
import os
warnings.filterwarnings("ignore")
from langchain.tools import DuckDuckGoSearchRun
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain

def get_stock_ticker(query):
    template = """You are a helpful assistant who extract company name from the human input.Please only output the company"""
    human_template = "{text}"

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("human", human_template),
    ])

    llm_chain = LLMChain(
        llm=llm,
        prompt=chat_prompt
    )

    company_name=llm_chain(query)['text'].strip()
    company_ticker = db_chain("\n\nHuman: What is the ticker symbol for " + str(company_name) + " in stock ticker table? \n\nAssistant:")
    return company_name, company_ticker['result']

def get_stock_price(ticker, history=10):
    print (ticker)
    today = date.today()
    start_date = today - timedelta(days=history)
    data = pdr.get_data_yahoo(ticker, start=start_date, end=today)
    dataname= ticker+'_'+str(today)
    return data, dataname

# Fetch top 5 google news for given company name
def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)

    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news

def stock_news_search(company_name):
    search=DuckDuckGoSearchRun()
    return search("Stock news about " + company_name)


# Get financial statements from Yahoo Finance
def get_financial_statements(ticker):
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]    # Only captures last 3 years of data
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    return balance_sheet

def anazlyze_stock(query):
    company_name,ticker=get_stock_ticker(query)
    print({"Query":query,"company_name":company_name,"Ticker":ticker})
    stock_data=get_stock_price(ticker,history=10)
    stock_financials=get_financial_statements(ticker)
    stock_news=get_recent_stock_news(company_name)

    available_information=f"Stock Price: {stock_data}\n\nStock Financials: {stock_financials}\n\nStock News: {stock_news}"
    analysis=llm(f"""\n\nHuman: Give detail stock analysis, Use the available data and provide investment recommendation. \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             You have the following information available about {company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
              {available_information} \
                \n\nAssistant: """
             )
    return analysis

In [ ]:
company_name, company_ticker = get_stock_ticker("What is the main business of Amazon?")

In [ ]:
print(company_name)
print(company_ticker)

In [ ]:
print(get_stock_price("AMZN"))

In [ ]:
print(get_recent_stock_news("Amazon"))


In [ ]:
print(get_financial_statements("AMZN"))

In [ ]:
stock_news_search("Amazon")

In [ ]:
analyze_result=anazlyze_stock("Is Amazon a good investment choice right now?")

In [ ]:
print(analyze_result)

## Part 4 - Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

In [ ]:
llm = BedrockModelWrapper(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs=model_parameter)

In [ ]:
# Making tool list

tools=[
    Tool(
        name="get company ticker",
        func=get_stock_ticker,
        description="Get the company stock ticker"
    ),
    Tool(
        name="get stock data",
        func=get_stock_price,
        description="Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it "
    ),
    Tool(
        name="get recent news",
        func=get_recent_stock_news,
        description="Use this to fetch recent news about stocks"
    ),

    Tool(
        name="get financial statements",
        func=get_financial_statements,
        description="Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it"
    ) 


]

In [ ]:
# Modify prompt to using Human: tag, and add predefine evaluation steps

updated_prompt="""Human: You are a financial advisor. Give stock recommendations for given query based on following instructions. 
<instructions>
Answer the following questions as best you can. You have access to the following tools:

get company ticker: Use when you need to extract company name and stock ticker. This tool will output company name and stock ticker. You should input the human input to it.
get stock data: Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the stock ticker to it.
get recent news: Use this to fetch recent news about stocks. This will output company stock news. You should innput the company name to it.
get financial statements: Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it
</instructions>

<steps>
Note- if you fail in satisfying any of the step below, Just move to next one
1) Use "get company ticker" tool to get the company name and stock ticker. Output- company name and stock ticker
2) Use "get stock data" tool to gather stock info. Output- Stock data
3) Use "get recent news" tool to search for latest stock realted news. Output- Stock news
4) Use "get financial statements" tool to get company's historic financial data. Output- Financial statement
5) Analyze the stock based on gathered data and give detail analysis for investment choice. provide numbers and reasons to justify your answer. Output- Detailed stock Analysis
</steps>

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [get company ticker, get stock data, get recent news, get financial statements]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}

"""


In [ ]:
from langchain.agents import initialize_agent 

zero_shot_agent=initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=True,
    max_iteration=2,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
)

zero_shot_agent.agent.llm_chain.prompt.template=updated_prompt

In [ ]:
response = zero_shot_agent("\n\nHuman: Is Tesla a good investment choice right now? \n\nAssistant:")

In [ ]:
# print output
print(response["output"])

In [ ]:
#print intermediate steps
print(response["intermediate_steps"])

In [ ]:
response_amazon = zero_shot_agent("\n\nHuman: Is Amazon a good investment choice right now? \n\nAssistant:")

In [ ]:
print(response_amazon["output"])

In [ ]:
print(response_amazon["intermediate_steps"])